# Part b) Resampling techniques, adding more complexity
In this notebook, we will use different techniques to ensure that our evaluation will be done on "fresh" data, rather than on the same data that was used for training.

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
os.sys.path.append(os.path.dirname(os.path.abspath('.')))

# Import local modules
from src.data.generate_data import FrankeFunction
from src.models.models import OLS
from src.evaluation.evaluation import mse, r_squared
from src.resampling.resampling import K_fold_splitter

In [2]:
df_X = pd.read_csv('../data/generated/X.csv', index_col=0)
df_z_no_noise = pd.read_csv('../data/generated/no_noise.csv', usecols=[1])
df_z_some_noise = pd.read_csv('../data/generated/some_noise.csv', usecols=[1])
df_z_noisy = pd.read_csv('../data/generated/noisy.csv', usecols=[1])

X = np.array(df_X)
z_no_noise = np.array(df_z_no_noise).ravel()
z_some_noise = np.array(df_z_some_noise).ravel()
z_noisy = np.array(df_z_noisy).ravel()

We will now use $\texttt{train_test_split}$ from $\texttt{sklearn.model_selection}$ in order to split our datasets in train and test sets. As we have three separate sets of target values, we will do this thrice.

In [3]:
targets = [{
    'name': 'No noise',
    'values': z_no_noise
},
{
    'name': 'Some noise (sigma 0.1)',
    'values': z_some_noise
},
{
    'name': 'Noisy (sigma 0.4)',
    'values': z_noisy
}]
col_names = ['MSE', 'R^2']
output_df = pd.DataFrame(columns=col_names)

print('%-30s|%-10s|%-10s' %('Data', 'MSE', 'R^2'))
print('-'*50)

for target in targets:
    X_train, X_test, z_train, z_test = train_test_split(X, target['values'], test_size=0.3, random_state=12)
    ol = OLS()
    ol.fit(X_train, z_train)
    predictions = ol.predict(X_test)
    mse_value = mse(z_test, predictions)
    r_2_value = r_squared(z_test, predictions)
    print('%-30s|%-10f|%-10f' %(target['name'], mse_value, r_2_value))
    output_df = output_df.append(pd.DataFrame(data=[[mse_value, r_2_value]], columns=col_names, index=[target['name']]))

output_df.to_csv('../reports/csv_files/2_mse_r2_score.csv')

Data                          |MSE       |R^2       
--------------------------------------------------
No noise                      |0.003428  |0.955624  
Some noise (sigma 0.1)        |0.014821  |0.822347  
Noisy (sigma 0.4)             |0.209289  |0.115033  


We will now use k-fold cross-validation to evaluate the test MSE score. 

In [10]:
k = 10
targets = [{
    'name': 'No noise',
    'values': z_no_noise
},
{
    'name': 'Some noise (sigma 0.1)',
    'values': z_some_noise
},
{
    'name': 'Noisy (sigma 0.4)',
    'values': z_noisy
}]

col_names = ['MSE']
output_df = pd.DataFrame(columns=col_names)
print('%-30s|%-10s' %('', 'MSE'))

print('-'*50)

for target in targets:
    mse_values = np.zeros(k)
    z = target['values']
    ol = OLS()
    
    kfs = K_fold_splitter(X.shape[0], k)
    fold_indices = kfs.cross_val_split()
    
    for j, fold in enumerate(fold_indices):     
        X_train = X[fold['train_indices']]
        z_train = z[fold['train_indices']]
        X_test = X[fold['test_indices']]
        z_test = z[fold['test_indices']]
        
        ol.fit(X_train, z_train)
        predictions = ol.predict(X_test)
        mean_squared_error = mse(z_test, predictions)
        
        mse_values[j] = mean_squared_error
    
    mse_value = np.mean(mse_values)
    print('%-30s|%-10f' %(target['name'], mse_value))
    output_df = output_df.append(pd.DataFrame(data=[[mse_value]], columns=col_names, index=[target['name']]))
output_df.to_csv('../reports/csv_files/2_cross_val_mse.csv')

                              |MSE       
--------------------------------------------------
No noise                      |0.002643  
Some noise (sigma 0.1)        |0.012347  
Noisy (sigma 0.4)             |0.171564  
